In [1]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

c:\Users\V Nikhil\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the Bayesian Network structure
model = BayesianNetwork([
    ('T', 'S'),  # Theft causes Siren
    ('Q', 'S'),  # Earthquake causes Siren
    ('S', 'K'),  # Siren causes Kevin to call
    ('S', 'L')   # Siren causes Lisa to call
])

In [3]:
data = pd.DataFrame(np.random.randint(0, 2, size=(1000, 5)), columns=['T', 'Q', 'S', 'K', 'L'])

#Learn the parameters (CPDs) from data
model.fit(data, estimator=MaximumLikelihoodEstimator)

In [4]:
#Print the learned CPDs
for cpd in model.get_cpds():
    print(f"\nCPD of {cpd.variable}:")
    print(cpd)


CPD of T:
+------+-------+
| T(0) | 0.497 |
+------+-------+
| T(1) | 0.503 |
+------+-------+

CPD of S:
+------+--------------------+-----+--------------------+
| Q    | Q(0)               | ... | Q(1)               |
+------+--------------------+-----+--------------------+
| T    | T(0)               | ... | T(1)               |
+------+--------------------+-----+--------------------+
| S(0) | 0.5148936170212766 | ... | 0.5471698113207547 |
+------+--------------------+-----+--------------------+
| S(1) | 0.4851063829787234 | ... | 0.4528301886792453 |
+------+--------------------+-----+--------------------+

CPD of Q:
+------+-------+
| Q(0) | 0.473 |
+------+-------+
| Q(1) | 0.527 |
+------+-------+

CPD of K:
+------+--------------------+--------------------+
| S    | S(0)               | S(1)               |
+------+--------------------+--------------------+
| K(0) | 0.4689922480620155 | 0.5144628099173554 |
+------+--------------------+--------------------+
| K(1) | 0.5310077

In [5]:
#Perform inference with the learned model
inference = VariableElimination(model)
result = inference.query(variables=['T'], evidence={'K': 1, 'L': 1})

print("\nP(T | K=true, L=true):")
print(result)


P(T | K=true, L=true):
+------+----------+
| T    |   phi(T) |
+======+==========+
| T(0) |   0.4969 |
+------+----------+
| T(1) |   0.5031 |
+------+----------+
